# 🧰 functools.wraps: Why You Need It in Every Proper Decorator

When you write a function decorator, you're **wrapping** one function inside another.  
But if you don't do it carefully, you **lose important metadata** about the original function — like its name, docstring, annotations, etc.

This is where `functools.wraps` comes in.

---

## 🤕 The Problem Without `wraps`

Let’s look at a basic decorator **without** using `wraps`:

In [7]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Calling decorated function")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(name):
    """Says hello to the user."""
    print(f"Hello, {name}!")

Now check the function:

In [8]:
print(greet.__name__)      # Output: wrapper ❌
print(greet.__doc__)       # Output: None ❌

wrapper
None


We lost everything about `greet`! Python thinks the function is called `wrapper` now.

---

## ✅ The Solution: `@wraps`

Python provides a decorator called `wraps()` in the `functools` module that **copies the metadata** from the original function to the wrapper.

In [10]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("Calling decorated function")
        return func(*args, **kwargs)
    return wrapper

Now when we decorate a function:

In [12]:
@my_decorator
def greet(name):
    """Says hello to the user."""
    print(f"Hello, {name}!")
    

# And check again:

print(greet.__name__)   # Output: greet ✅
print(greet.__doc__)    # Output: Says hello to the user. ✅


greet
Says hello to the user.


---

## 📦 What `@wraps` Preserves

| Attribute         | Description                     |
|-------------------|---------------------------------|
| `__name__`        | Original function name          |
| `__doc__`         | Docstring                      |
| `__annotations__` | Type hints                      |
| `__module__`      | Module where it was defined     |
| `__dict__`        | Custom attributes if any        |

Internally, `@wraps` is just:

```python
def wraps(func):
    def decorator(wrapper):
        wrapper.__name__ = func.__name__
        wrapper.__doc__ = func.__doc__
        ...
        return wrapper
    return decorator
```

---

## ✅ Summary

- Always use `@wraps` when writing decorators.
- It keeps your decorated function's identity intact.
- Without it, tools like `help()`, `inspect`, and even `debuggers` get confused.

---

> 🧠 Pro Tip: If you're chaining multiple decorators, **each one** should use `@wraps`.



# 🎛️ Decorators with Arguments: Understanding the Function Call Flow

When you see a decorator written like this:

```python
@log(level="DEBUG")
def greet(name):
    print(f"Hello, {name}!")
```

You're not just applying a decorator — you're actually **calling a function** and then using its return value as a decorator.

---

## 🧠 What’s Really Happening?

This line:

```python
@log(level="DEBUG")
```

is equivalent to this:

```python
my_decorator = log(level="DEBUG")
greet = my_decorator(greet)
```

So when Python executes this code:
1. It calls `log(level="DEBUG")`
2. This returns a **decorator** (a function that takes `greet` as input)
3. That decorator is then applied to `greet`

---


## 🛠 So How Do We Write `log`?

Here’s how to define a decorator that **accepts arguments**:

In [16]:
from functools import wraps

def log(level="INFO"):
    def actual_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"[{level}] Calling {func.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return actual_decorator

---

## 🔁 Flow of Execution

| Step                        | What Happens                              |
|-----------------------------|--------------------------------------------|
| `log(level="DEBUG")`        | Called immediately, returns a decorator    |
| `@log(...)`                 | Same as `greet = decorator(greet)`         |
| `wrapper(*args, **kwargs)`  | Executes when `greet()` is called          |

---

## 🧪 Usage Example


In [17]:
@log(level="DEBUG")
def greet(name):
    print(f"Hello, {name}!")

greet("Alice")

[DEBUG] Calling greet
Hello, Alice!



---

## 🧠 Summary

- `@decorator(args)` is a **function call**, not just a label
- It must return a real decorator: a function that takes another function
- This is how decorators like `@retry(times=3)` or `@timeout(seconds=10)` work

---

Next, we’ll build our own `@repeat(n=3)` decorator that runs a function multiple times 🔁


# 🧠 Class-Based Decorators with Functors (Function Objects)

In Python, any object that defines the special method `__call__()` is **callable** — meaning you can use it like a function.

Such objects are known as **functors**, or **function objects**.

This means you can use a **class** as a decorator, by making it behave like a function using `__call__`.

---

## ✅ Why Use a Class Instead of a Function?

Class-based decorators are useful when:

- You need to **maintain internal state**
- You want better **structure** and **encapsulation**
- You’re working with **configurable decorators**
- You want decorators to be more **testable** and **extensible**

---

## 🧰 Example: A Call Counter Decorator

We’ll create a decorator that **counts how many times a function has been called**.


In [18]:
class CallCounter:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"[{self.func.__name__}] has been called {self.count} time(s).")
        return self.func(*args, **kwargs)


---

### 🔍 How It Works

- `__init__()` receives the original function and stores it
- `__call__()` wraps the original function and adds behavior
- The class instance **acts as a decorator** because it is callable

---

## 🧪 Usage

In [29]:
@CallCounter
def say_hello():
    print("Hello!")

say_hello()
say_hello()
say_hello()

[say_hello] has been called 1 time(s).
Hello!
[say_hello] has been called 2 time(s).
Hello!
[say_hello] has been called 3 time(s).
Hello!


---

## 🧠 Real-World Use Cases

- Rate limiters
- Retry mechanisms
- Access logging
- Monitoring & metrics
- Analytics tracking

---

## 🔄 Comparison: Function vs Class Decorators

| Feature             | Function-based        | Class-based           |
|---------------------|-----------------------|------------------------|
| Easy to write       | ✅                    | ✅                     |
| Holds internal state| ❌ (unless using closures) | ✅                 |
| Better structure    | ❌                    | ✅                     |
| Good for config     | 🔸 (with nesting)     | ✅                     |
| Reusability         | 🔸                    | ✅                     |

---

## ✅ Summary

- A **functor** is any object with a `__call__` method.
- You can use a class as a decorator by defining `__init__` and `__call__`.
- This allows decorators to store state, be more modular, and scale better in large applications.

---

> ✨ If you need powerful decorators with memory, flexibility, or configuration — class-based decorators are the cleanest choice.


# 🏗️ Parameterized Class-Based Decorators in Python

In the previous lesson, we learned how to build a class-based decorator using a callable class (a **functor**).

Now let’s take it to the next level:
We’ll build a class-based decorator that also **accepts arguments**, like this:

```python
@Track(enabled=True)
def greet(name):
    print(f"Hello, {name}!")
```

---

## 🧠 What’s Really Happening?

When Python sees `@Track(enabled=True)`, it does this:

```python
decorator = Track(enabled=True)  # Create instance of decorator class
greet = decorator(greet)         # Call the instance with the target function
```

So your class must:
1. Accept arguments in `__init__`
2. Be **callable** (implement `__call__`)
3. Apply the actual decoration inside `__call__`

---


## 🧰 Example: A Toggleable Tracker

We’ll build a class-based decorator that logs a message **only if enabled**.

In [26]:
from functools import wraps

class Track:
    def __init__(self, enabled=True):
        self.enabled = enabled

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if self.enabled:
                print(f"[TRACK] Calling {func.__name__}")
            return func(*args, **kwargs)
        return wrapper

---

## 🧪 Usage


In [30]:
@Track(enabled=True)
def say_hello():
    print("Hello!")

@Track(enabled=False)
def say_hi():
    print("Hi!")

say_hello()
say_hi()

[TRACK] Calling say_hello
Hello!
Hi!


---

## 🧩 Breakdown of the Pattern

| Method         | Role                                       |
|----------------|--------------------------------------------|
| `__init__()`   | Accepts decorator arguments                |
| `__call__()`   | Receives the target function to decorate   |
| `wrapper()`    | The actual logic that wraps the function   |

---

## 🧠 Best Practices

- Always use `@wraps(func)` to preserve metadata
- You can store config values like `enabled`, `level`, `retry_count`, etc.
- Useful for **feature toggles**, **logging**, **validation**, and more

---

## ✅ Summary

- A **parameterized class-based decorator** is just a callable class that takes arguments.
- This gives you the **clarity of class-based design** + the **flexibility of configurable decorators**.
- You can now build decorators that behave differently based on external input.

---

> 💬 Want to go even further? You can combine class decorators with inheritance or build reusable decorator **mixins** for tracking, logging, caching, and more.

Next up: Let’s build `@Retry(n=3)` — a decorator that automatically re-executes a function if it raises an error.


# 💥 Example: Building `@Retry(n=3)` — A Fault-Tolerant Decorator

When writing production code, it's common to deal with functions that may fail due to **temporary issues** — network errors, timeouts, or unavailable services.

One practical solution is to use a **retry decorator**:  
It automatically re-executes the function a fixed number of times if an exception occurs.

---

## 🎯 Goal

Build a decorator that allows you to write:

```python
@Retry(n=3)
def risky_operation():
    ...
```

If `risky_operation()` raises an error, it will retry up to `n` times before finally giving up.

---

## 🧠 What It Should Do

- Accept a parameter `n` = number of retries
- Wrap the function using `__call__`
- On exception, **retry** the function `n` times
- Optionally print a message when retrying

---

## 🧰 Implementation


In [31]:
from functools import wraps
import time

class Retry:
    def __init__(self, n=3):
        self.n = n

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < self.n:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    print(f"[Retry {attempts}/{self.n}] {func.__name__} failed: {e}")
                    time.sleep(0.5)  # Optional: wait before retry
            raise RuntimeError(f"{func.__name__} failed after {self.n} retries.")
        return wrapper


---

## 🧪 Usage


In [36]:
import random
from rich.console import Console

console = Console()

@Retry(n=5)
def unstable_function():
    if random.random() < 0.7:
        raise ValueError("Something went wrong!")
    print("Success!")

try:
    unstable_function()
except:
    console.print_exception()

[Retry 1/5] unstable_function failed: Something went wrong!
[Retry 2/5] unstable_function failed: Something went wrong!
[Retry 3/5] unstable_function failed: Something went wrong!
[Retry 4/5] unstable_function failed: Something went wrong!
[Retry 5/5] unstable_function failed: Something went wrong!


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   print("Success!")                                                                       │
│   11                                                                                             │
│   12 try:                                                                                        │
│ ❱ 13 │   unstable_function()                                                                     │
│   14 except:                                                                                     │
│   15 │   console.print_exception()                                                               │
│   16                                                                                             │
│                                                                                                  │
│ in wrapper:19                                                                                    │
│                                                                                                  │
│   16 │   │   │   │   │   attempts += 1                                                           │
│   17 │   │   │   │   │   print(f"[Retry {attempts}/{self.n}] {func.__name__} failed: {e}")       │
│   18 │   │   │   │   │   time.sleep(0.5)  # Optional: wait before retry                          │
│ ❱ 19 │   │   │   raise RuntimeError(f"{func.__name__} failed after {self.n} retries.")           │
│   20 │   │   return wrapper                                                                      │
│   21                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: unstable_function failed after 5 retries.

---

## 🧠 Why This Is Powerful

- It helps handle **intermittent errors** without crashing the program
- You can use it in:
  - Network/API requests
  - File system operations
  - Database connections

---

## 🧾 Summary

| Feature        | Description                                  |
|----------------|----------------------------------------------|
| Class-based    | Stores retry count in `self.n`               |
| Parameterized  | Can call like `@Retry(n=5)`                  |
| Exception-safe | Handles failures and retries automatically   |
| Reusable       | Plug it into any risky function              |

---

> 🔧 You’ve just built your own **resilience layer** using a clean, reusable decorator. This is a real-world pattern used in many Python frameworks and systems.
